# This notebook is used for understand the covarience between the population of melbourne and rental affordability

For the sake of simplicty, this notebook contains the following
- Livability scores of melbourne
- Rental affordability of greater Melbourne

In [4]:
# Load libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
from statsmodels.stats.api import anova_lm
from scipy.stats import pearsonr
import statsmodels.api as sm
import plotly.express as px
import plotly.graph_objs as go


## Data processing

The below data will be used for this model:

### Response variable: Regional population dataset

This dataset contains the population related information in Victoria, ranging from births, deaths to migration.
The key feature that we're interested in is the erp_YYYY column, which holds the estimated regional population for Melbourne

### Predictor variable: Greater Melbourne rental affordability index

- This dataset contains information about the rental affordability 

## Initial data analysis

In [5]:
# Load data

rental_affordability = pd.read_csv("../data/response/rental_affordability.csv")
rental_affordability.columns = rental_affordability.columns.str.replace(' ', '')

response = pd.read_csv("../data/response/abs_regional_population_lga_2001_2021.csv")

 Now we proceed to look at rental_affordability

In [6]:
rental_affordability

,geography_name,rai_cityadjusted_total_2011_q2,rai_cityadjusted_total_2011_q3,rai_cityadjusted_total_2020_q1,rai_cityadjusted_total_2011_q1,state,rai_cityadjusted_total_2020_q3,rai_cityadjusted_total_2019_q4,rai_cityadjusted_total_2020_q2,rai_cityadjusted_total_2011_q4,...,rai_cityadjusted_total_2012_q1,rai_cityadjusted_total_2015_q3,rai_cityadjusted_total_2012_q4,rai_cityadjusted_total_2014_q3,rai_cityadjusted_total_2012_q3,rai_cityadjusted_total_2014_q4,rai_cityadjusted_total_2015_q1,rai_cityadjusted_total_2014_q1,rai_cityadjusted_total_2014_q2,rai_cityadjusted_total_2015_q4
0,3000,98.33916,98.33916,109.61540,93.91771,VIC,144.2308,109.61540,137.0192,100.62610,...,97.23423,102.56410,103.71650,107.33450,96.15385,107.33450,102.79250,102.56410,102.56410,114.04290
1,3003,97.23423,100.62610,107.46610,88.75740,VIC,144.2308,109.61540,121.7949,81.64006,...,94.06355,103.71650,100.33440,107.33450,90.14423,102.56410,102.56410,113.96010,99.25558,119.60600
2,3004,89.21491,88.30455,103.41070,84.13462,VIC,130.3781,105.39940,111.8524,99.46950,...,86.53846,96.15385,94.19152,98.19967,90.14423,97.16599,96.15385,102.56410,97.16599,102.16350
3,3005,NaN,NaN,NaN,NaN,VIC,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3006,81.64006,78.67133,99.65035,77.66272,VIC,144.2308,99.65035,121.7949,83.21006,...,81.64006,87.91209,88.75740,88.75740,81.64006,90.49774,87.08273,90.49774,90.49774,98.07692
5,3008,77.26648,83.21006,95.31773,74.78632,VIC,135.7466,97.87088,105.3994,79.39308,...,82.41758,90.49774,92.30769,89.61912,88.30455,92.30769,90.49774,93.24009,94.19152,96.15385


The dataset contains rental affordability index (RAI) values for different postal codes in Melbourne over multiple quarters in each year from 2011. To find the rental affordability for Melbourne as a whole, we will calculate the average RAI for each year .

In [41]:


# Strip leading and trailing spaces from column names
rental_affordability.columns = rental_affordability.columns.str.strip()

# Define the yearly columns
yearly_columns = {
    '2011': ['rai_cityadjusted_total_2011_q1', 'rai_cityadjusted_total_2011_q2', 'rai_cityadjusted_total_2011_q3', 'rai_cityadjusted_total_2011_q4'],
    '2012': ['rai_cityadjusted_total_2012_q1', 'rai_cityadjusted_total_2012_q2', 'rai_cityadjusted_total_2012_q3', 'rai_cityadjusted_total_2012_q4'],
    '2013': ['rai_cityadjusted_total_2013_q1', 'rai_cityadjusted_total_2013_q2', 'rai_cityadjusted_total_2013_q3', 'rai_cityadjusted_total_2013_q4'],
    '2014': ['rai_cityadjusted_total_2014_q1', 'rai_cityadjusted_total_2014_q2', 'rai_cityadjusted_total_2014_q3', 'rai_cityadjusted_total_2014_q4'],
    '2015': ['rai_cityadjusted_total_2015_q1', 'rai_cityadjusted_total_2015_q2', 'rai_cityadjusted_total_2015_q3', 'rai_cityadjusted_total_2015_q4'],
    '2016': ['rai_cityadjusted_total_2016_q1', 'rai_cityadjusted_total_2016_q2', 'rai_cityadjusted_total_2016_q3', 'rai_cityadjusted_total_2016_q4'],
    '2017': ['rai_cityadjusted_total_2017_q1', 'rai_cityadjusted_total_2017_q2', 'rai_cityadjusted_total_2017_q3', 'rai_cityadjusted_total_2017_q4'],
    '2018': ['rai_cityadjusted_total_2018_q1', 'rai_cityadjusted_total_2018_q2', 'rai_cityadjusted_total_2018_q3', 'rai_cityadjusted_total_2018_q4'],
    '2019': ['rai_cityadjusted_total_2019_q1', 'rai_cityadjusted_total_2019_q2', 'rai_cityadjusted_total_2019_q3', 'rai_cityadjusted_total_2019_q4'],
    '2020': ['rai_cityadjusted_total_2020_q1', 'rai_cityadjusted_total_2020_q2', 'rai_cityadjusted_total_2020_q3', 'rai_cityadjusted_total_2020_q4'],
    '2021': ['rai_cityadjusted_total_2021_q1', 'rai_cityadjusted_total_2021_q2', 'rai_cityadjusted_total_2021_q3', 'rai_cityadjusted_total_2021_q4']
}

# Calculate the average RAI for each year
yearly_avg_rai = {}

for year, cols in yearly_columns.items():
    available_cols = [col for col in cols if col in rental_affordability.columns]
    yearly_avg_rai[year] = rental_affordability[available_cols].mean(axis=1).mean()

# Convert to DataFrame 
rental_affordability_clean = pd.DataFrame.from_dict(yearly_avg_rai, orient='index', columns=['rental_affordability_index']).reset_index()
rental_affordability_clean.rename(columns={'index': 'year'}, inplace=True)

# change type for merging with other dataset

rental_affordability_clean['year'] = rental_affordability_clean['year'].astype(int)



In [42]:
rental_affordability_clean

,year,rental_affordability_index
0,2011,87.822180
1,2012,89.982100
2,2013,97.576380
3,2014,98.380814
4,2015,98.367963
5,2016,100.701249
6,2017,103.080537
7,2018,102.548016
8,2019,103.314577
9,2020,127.751459


Cleaning up of response dataset and retriving data of region = melbourne

In [43]:
# Remove spacing from column names

response.columns = response.columns.str.replace(' ', '')

In [44]:
# Get all key columns for our response dataset. The years range from 2011 to 2021, inclusive of start & end. 

KEY_RESPONSE_COLUMNS = ['lga_name_2021']

for i in range(2011, 2022):
    key_column = f"erp_{i}"
    KEY_RESPONSE_COLUMNS.append(key_column)
    

In [45]:
response[KEY_RESPONSE_COLUMNS]

,lga_name_2021,erp_2011,erp_2012,erp_2013,erp_2014,erp_2015,erp_2016,erp_2017,erp_2018,erp_2019,erp_2020,erp_2021
0,Banyule,122983,123584,124314,125107,126088,127447,129115,130250,131640,131895,129387
1,Bayside (Vic.),96119,96971,98252,99797,101268,102912,104282,105745,106856,107516,105580
2,Boroondara,167062,168483,170485,172660,174753,177276,179677,181376,183197,182974,176632
3,Darebin,142942,144986,147272,149511,151974,155126,158751,161653,164224,166356,162501
4,Glen Eira,137152,139116,141260,143694,145924,148583,151457,153920,156535,158149,155123
5,Hobsons Bay,87395,88339,89188,90417,91674,93445,95071,96481,97765,98146,96317
6,Maribyrnong,75154,77179,79782,82463,84783,86942,89362,91413,93482,94928,93467
7,Melbourne,100240,107578,118711,127975,136872,146096,159141,170317,178994,183628,169860
8,Moonee Valley,112180,113673,115464,117763,120231,122871,125411,127906,130313,131702,129379
9,Moreland,154247,156967,160282,164099,167870,172294,176974,181780,185811,188660,184707


In [46]:
# Pick out row 7 to turn into dataframe ( melbourne)

response_clean = pd.DataFrame(response[KEY_RESPONSE_COLUMNS].iloc[7]).reset_index()

In [47]:
# Rename and clean columns

response_clean = response_clean.rename(columns={'index': 'year', 7: 'population'})
response_clean = response_clean.iloc[1:] # Remove unnceessary row
response_clean['year'] = response_clean['year'].str.removeprefix('erp_')
response_clean['year'] = response_clean['year'].astype(int)

In [48]:
response_clean

,year,population
1,2011,100240
2,2012,107578
3,2013,118711
4,2014,127975
5,2015,136872
6,2016,146096
7,2017,159141
8,2018,170317
9,2019,178994
10,2020,183628


merge the 2 dataset into a dataframe for analysis : analysis_df

In [49]:
analysis_df = pd.merge(rental_affordability_clean, response_clean, on='year')
analysis_df['year'] = pd.to_numeric(analysis_df['year'])
analysis_df['rental_affordability_index'] = pd.to_numeric(analysis_df['rental_affordability_index'])
analysis_df['population'] = pd.to_numeric(analysis_df['population'])

In [50]:
analysis_df

,year,rental_affordability_index,population
0,2011,87.822180,100240
1,2012,89.982100,107578
2,2013,97.576380,118711
3,2014,98.380814,127975
4,2015,98.367963,136872
5,2016,100.701249,146096
6,2017,103.080537,159141
7,2018,102.548016,170317
8,2019,103.314577,178994
9,2020,127.751459,183628
